In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
from utils import *
import pandas as pd

In [3]:
df = pd.read_csv('../data/parsed_data.csv')
df = df.drop('Unnamed: 0', axis=1)

df

,id,title,area,interpretability,doi,source
0,797,Restricted Recurrent Neural Tensor Networks: E...,NaN,NaN,10.18653/v1/P18-2002,ACL2019
1,338,Neural Hidden Markov Model for Machine Transla...,NaN,NaN,10.18653/v1/P18-2060,ACL2019
2,300,Attention Focusing for Neural Machine Translat...,NaN,NaN,10.18653/v1/P18-1164,ACL2019
3,1638,Will it Blend? Blending Weak and Strong Labele...,NaN,NaN,10.18653/v1/P18-2095,ACL2019
4,92,Batch IS NOT Heavy: Learning Word Representati...,NaN,NaN,10.18653/v1/P18-1172,ACL2019
...,...,...,...,...,...,...
9971,1125,Know What You Don’t Know: Unanswerable Questio...,Best Paper Session,NaN,10.18653/v1/P18-2124,ACL2018
9972,1603,‘Lighter’ Can Still Be Dark: Modeling Comparat...,Best Paper Session,NaN,10.18653/v1/P18-2125,ACL2018
9973,618,Finding syntax in human encephalography with b...,Best Paper Session,NaN,10.18653/v1/P18-1254,ACL2018
9974,1247,Learning to Ask Good Questions: Ranking Clarif...,Best Paper Session,NaN,10.18653/v1/P18-1255,ACL2018


In [4]:
df_with_dois = df[df['doi'].notna()]

df_with_dois

,id,title,area,interpretability,doi,source
0,797,Restricted Recurrent Neural Tensor Networks: E...,NaN,NaN,10.18653/v1/P18-2002,ACL2019
1,338,Neural Hidden Markov Model for Machine Transla...,NaN,NaN,10.18653/v1/P18-2060,ACL2019
2,300,Attention Focusing for Neural Machine Translat...,NaN,NaN,10.18653/v1/P18-1164,ACL2019
3,1638,Will it Blend? Blending Weak and Strong Labele...,NaN,NaN,10.18653/v1/P18-2095,ACL2019
4,92,Batch IS NOT Heavy: Learning Word Representati...,NaN,NaN,10.18653/v1/P18-1172,ACL2019
...,...,...,...,...,...,...
9971,1125,Know What You Don’t Know: Unanswerable Questio...,Best Paper Session,NaN,10.18653/v1/P18-2124,ACL2018
9972,1603,‘Lighter’ Can Still Be Dark: Modeling Comparat...,Best Paper Session,NaN,10.18653/v1/P18-2125,ACL2018
9973,618,Finding syntax in human encephalography with b...,Best Paper Session,NaN,10.18653/v1/P18-1254,ACL2018
9974,1247,Learning to Ask Good Questions: Ranking Clarif...,Best Paper Session,NaN,10.18653/v1/P18-1255,ACL2018


In [5]:
print('papers missing doi', len(df) - len(df_with_dois))

papers missing doi 58


In [6]:
df_with_interpretability_boolean = df_with_dois[df_with_dois['interpretability'].notna()] # we could not determine if some of them are from interpretability
df_with_interpretability_boolean

,id,title,area,interpretability,doi,source
844,NaN,How Does Counterfactually Augmented Data Impac...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.emnlp-main.28,EMNLP2021
845,NaN,Mining the Cause of Political Decision-Making ...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.27,EMNLP2021
846,NaN,WHOSe Heritage: Classification of UNESCO World...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.34,EMNLP2021
847,NaN,To Protect and To Serve? Analyzing Entity-Cent...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.82,EMNLP2021
848,NaN,An unsupervised framework for tracing textual ...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.105,EMNLP2021
...,...,...,...,...,...,...
9524,T4773,Rank-Aware Negative Training for Semi-Supervis...,Machine Learning for NLP,False,10.1162/tacl_a_00574,ACL2023
9525,T4777,Transparency Helps Reveal When Language Models...,"Linguistic Theories, Cognitive Modeling, and P...",False,10.1162/tacl_a_00565,ACL2023
9526,T4803,Design Choices for Crowdsourcing Implicit Disc...,Discourse and Pragmatics,False,10.1162/tacl_a_00586,ACL2023
9527,T4929,Time-and-Space-Efficient Weighted Deduction,"Semantics: Sentence-level Semantics, Textual I...",False,10.1162/tacl_a_00588,ACL2023


In [7]:
print('papers missing interpretability boolean', len(df_with_dois) - len(df_with_interpretability_boolean))

papers missing interpretability boolean 2052


In [8]:
df_with_interpretability_boolean['interpretability'].value_counts()

interpretability
False    7370
True      496
Name: count, dtype: int64

In [9]:
import requests
import bs4

def get_abstract(doi):
    url_path = doi.split('/')[-1]
    url = 'https://aclanthology.org/' + url_path
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    css_selector = "#main > div.row.acl-paper-details > div.col.col-lg-10.order-2 > div > div > span"
    element = soup.select_one(css_selector)
    abstract = element.text
    return abstract



In [ ]:
processed_df = df_with_interpretability_boolean.copy()
processed_df['abstract'] = None

In [58]:
import pandas as pd
import os
import time
from tqdm import tqdm

def fill_abstracts_being_fault_tolerant(df, save_path):
    if os.path.exists(save_path):
        df = pd.read_pickle(save_path)

    for i in tqdm(range(len(df))):
        if df.iloc[i]['doi'] is not None and pd.isna(df.iloc[i]['abstract']):
            try:
                abstract = get_abstract(df.iloc[i]['doi'])
                df.iloc[i, df.columns.get_loc('abstract')] = abstract
                df.to_pickle(save_path)
            except Exception as error:
                print(error)
                print(i, df.iloc[i]['doi'])
    return df


file_path = 'tmp.pkl'
processed_df = fill_abstracts_being_fault_tolerant(processed_df, file_path)
processed_df


 51%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                 | 3984/7866 [00:00<00:00, 39834.72it/s]

'NoneType' object has no attribute 'text'
4155 10.18653/v1/2022.emnlp-main.782


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7866/7866 [00:02<00:00, 3263.15it/s]

'NoneType' object has no attribute 'text'
4159 10.18653/v1/2022.emnlp-main.786


,id,title,area,interpretability,doi,source,abstract
844,NaN,How Does Counterfactually Augmented Data Impac...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.emnlp-main.28,EMNLP2021,As NLP models are increasingly deployed in soc...
845,NaN,Mining the Cause of Political Decision-Making ...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.27,EMNLP2021,Mining the causes of political decision-making...
846,NaN,WHOSe Heritage: Classification of UNESCO World...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.34,EMNLP2021,The UNESCO World Heritage List (WHL) includes ...
847,NaN,To Protect and To Serve? Analyzing Entity-Cent...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.82,EMNLP2021,Framing has significant but subtle effects on ...
848,NaN,An unsupervised framework for tracing textual ...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.105,EMNLP2021,Morality plays an important role in social wel...
...,...,...,...,...,...,...,...
9524,T4773,Rank-Aware Negative Training for Semi-Supervis...,Machine Learning for NLP,False,10.1162/tacl_a_00574,ACL2023,Semi-supervised text classification-based para...
9525,T4777,Transparency Helps Reveal When Language Models...,"Linguistic Theories, Cognitive Modeling, and P...",False,10.1162/tacl_a_00565,ACL2023,Many current NLP systems are built from langua...
9526,T4803,Design Choices for Crowdsourcing Implicit Disc...,Discourse and Pragmatics,False,10.1162/tacl_a_00586,ACL2023,Disagreement in natural language annotation ha...
9527,T4929,Time-and-Space-Efficient Weighted Deduction,"Semantics: Sentence-level Semantics, Textual I...",False,10.1162/tacl_a_00588,ACL2023,Many NLP algorithms have been described in ter...


In [83]:
def get_abstract_from_crossref(doi):
    url = 'https://api.crossref.org/works/' + doi
    r = requests.get(url)
    response = r.json()
    soup = bs4.BeautifulSoup(response['message']['abstract'], 'html.parser')
    return soup.find('jats:p').text


def fill_abstracts_crossref(df, save_path):
    if os.path.exists(save_path):
        df = pd.read_pickle(save_path)

    for i in tqdm(range(len(df))):
        if df.iloc[i]['doi'] is not None and pd.isna(df.iloc[i]['abstract']):
            try:
                abstract = get_abstract_from_crossref(df.iloc[i]['doi'])
                df.iloc[i, df.columns.get_loc('abstract')] = abstract
                df.to_pickle(save_path)
            except Exception as error:
                print(error)
                print(i, df.iloc[i]['doi'])
    return df

file_path = 'tmp.pkl'
processed_df = fill_abstracts_crossref(processed_df, file_path)
processed_df


 38%|██████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                                               | 2958/7866 [00:00<00:00, 29573.47it/s]

'abstract'
4155 10.18653/v1/2022.emnlp-main.782


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7866/7866 [00:01<00:00, 5112.61it/s]

'abstract'
4159 10.18653/v1/2022.emnlp-main.786


,id,title,area,interpretability,doi,source,abstract
844,NaN,How Does Counterfactually Augmented Data Impac...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.emnlp-main.28,EMNLP2021,As NLP models are increasingly deployed in soc...
845,NaN,Mining the Cause of Political Decision-Making ...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.27,EMNLP2021,Mining the causes of political decision-making...
846,NaN,WHOSe Heritage: Classification of UNESCO World...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.34,EMNLP2021,The UNESCO World Heritage List (WHL) includes ...
847,NaN,To Protect and To Serve? Analyzing Entity-Cent...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.82,EMNLP2021,Framing has significant but subtle effects on ...
848,NaN,An unsupervised framework for tracing textual ...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.105,EMNLP2021,Morality plays an important role in social wel...
...,...,...,...,...,...,...,...
9524,T4773,Rank-Aware Negative Training for Semi-Supervis...,Machine Learning for NLP,False,10.1162/tacl_a_00574,ACL2023,Semi-supervised text classification-based para...
9525,T4777,Transparency Helps Reveal When Language Models...,"Linguistic Theories, Cognitive Modeling, and P...",False,10.1162/tacl_a_00565,ACL2023,Many current NLP systems are built from langua...
9526,T4803,Design Choices for Crowdsourcing Implicit Disc...,Discourse and Pragmatics,False,10.1162/tacl_a_00586,ACL2023,Disagreement in natural language annotation ha...
9527,T4929,Time-and-Space-Efficient Weighted Deduction,"Semantics: Sentence-level Semantics, Textual I...",False,10.1162/tacl_a_00588,ACL2023,Many NLP algorithms have been described in ter...


In [91]:
def get_abstract_from_semantic_scholar(paper_id):
    query = f"https://api.semanticscholar.org/graph/v1/paper/{paper_id}"
    fields = "abstract"
    
    # query Semantic Scholar API
    response = requests.get(query, headers={"x-api-key": API_KEY}, params={"fields": fields})

    # Success! Convert response to dict 
    paper_dict = response.json()

    assert type(paper_dict['abstract']) == str
    return paper_dict['abstract']

def fill_abstracts_semantic_scholar(df, save_path):
    if os.path.exists(save_path):
        df = pd.read_pickle(save_path)

    for i in tqdm(range(len(df))):
        if df.iloc[i]['doi'] is not None and pd.isna(df.iloc[i]['abstract']):
            try:
                abstract = get_abstract_from_semantic_scholar(df.iloc[i]['doi'])
                df.iloc[i, df.columns.get_loc('abstract')] = abstract
                df.to_pickle(save_path)
            except Exception as error:
                print(error)
                print(i, df.iloc[i]['doi'])
    return df

file_path = 'tmp.pkl'
processed_df = fill_abstracts_semantic_scholar(processed_df, file_path)
processed_df


 53%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                             | 4156/7866 [00:00<00:00, 4790.88it/s]

'abstract'
4155 10.18653/v1/2022.emnlp-main.782


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7866/7866 [00:01<00:00, 4476.33it/s]

'abstract'
4159 10.18653/v1/2022.emnlp-main.786


,id,title,area,interpretability,doi,source,abstract
844,NaN,How Does Counterfactually Augmented Data Impac...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.emnlp-main.28,EMNLP2021,As NLP models are increasingly deployed in soc...
845,NaN,Mining the Cause of Political Decision-Making ...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.27,EMNLP2021,Mining the causes of political decision-making...
846,NaN,WHOSe Heritage: Classification of UNESCO World...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.34,EMNLP2021,The UNESCO World Heritage List (WHL) includes ...
847,NaN,To Protect and To Serve? Analyzing Entity-Cent...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.82,EMNLP2021,Framing has significant but subtle effects on ...
848,NaN,An unsupervised framework for tracing textual ...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.105,EMNLP2021,Morality plays an important role in social wel...
...,...,...,...,...,...,...,...
9524,T4773,Rank-Aware Negative Training for Semi-Supervis...,Machine Learning for NLP,False,10.1162/tacl_a_00574,ACL2023,Semi-supervised text classification-based para...
9525,T4777,Transparency Helps Reveal When Language Models...,"Linguistic Theories, Cognitive Modeling, and P...",False,10.1162/tacl_a_00565,ACL2023,Many current NLP systems are built from langua...
9526,T4803,Design Choices for Crowdsourcing Implicit Disc...,Discourse and Pragmatics,False,10.1162/tacl_a_00586,ACL2023,Disagreement in natural language annotation ha...
9527,T4929,Time-and-Space-Efficient Weighted Deduction,"Semantics: Sentence-level Semantics, Textual I...",False,10.1162/tacl_a_00588,ACL2023,Many NLP algorithms have been described in ter...


In [113]:
processed_df[processed_df['abstract'].isna()]

,id,title,area,interpretability,doi,source,abstract,embedding


In [109]:
### some papers did not have their abstract by any of the sources
### these abstracts where added manually

In [111]:
from transformers import AutoTokenizer
from adapters import AutoAdapterModel
from tqdm import tqdm
tqdm.pandas()

tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')

model = AutoAdapterModel.from_pretrained("allenai/specter2_base")
adapter_name = model.load_adapter("allenai/specter2_classification", source="hf", set_active=True)

def get_embedding(paper_row):
    text = paper_row['title'] + tokenizer.sep_token + paper_row['abstract']
    inputs = tokenizer(text,
                       padding=True,
                       truncation=True,
                       return_tensors="pt",
                       return_token_type_ids=False,
                       max_length=512)
    output = model(**inputs)
    embeddings = output.last_hidden_state[:, 0, :][0].detach().numpy()
    return embeddings

processed_df['embedding'] = processed_df.progress_apply(get_embedding, axis=1)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7866/7866 [12:55<00:00, 10.14it/s]


In [112]:
processed_df

,id,title,area,interpretability,doi,source,abstract,embedding
844,NaN,How Does Counterfactually Augmented Data Impac...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.emnlp-main.28,EMNLP2021,As NLP models are increasingly deployed in soc...,"[-0.7192544, 0.8835577, -0.48404795, -0.883667..."
845,NaN,Mining the Cause of Political Decision-Making ...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.27,EMNLP2021,Mining the causes of political decision-making...,"[-1.3761567, 0.7755194, -2.4345405, 0.24337131..."
846,NaN,WHOSe Heritage: Classification of UNESCO World...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.34,EMNLP2021,The UNESCO World Heritage List (WHL) includes ...,"[-1.2191356, -0.05772033, -0.48436293, -0.5805..."
847,NaN,To Protect and To Serve? Analyzing Entity-Cent...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.82,EMNLP2021,Framing has significant but subtle effects on ...,"[-0.3179136, 0.9814322, -2.040924, -0.9015022,..."
848,NaN,An unsupervised framework for tracing textual ...,Computational Social Science and Cultural Anal...,False,10.18653/v1/2021.findings-emnlp.105,EMNLP2021,Morality plays an important role in social wel...,"[-0.87162125, 0.13068287, -0.8506166, -1.16890..."
...,...,...,...,...,...,...,...,...
9524,T4773,Rank-Aware Negative Training for Semi-Supervis...,Machine Learning for NLP,False,10.1162/tacl_a_00574,ACL2023,Semi-supervised text classification-based para...,"[-0.7364854, -0.4441464, -0.6042715, -1.049953..."
9525,T4777,Transparency Helps Reveal When Language Models...,"Linguistic Theories, Cognitive Modeling, and P...",False,10.1162/tacl_a_00565,ACL2023,Many current NLP systems are built from langua...,"[-0.29979014, 0.25522092, -0.17783862, -1.1191..."
9526,T4803,Design Choices for Crowdsourcing Implicit Disc...,Discourse and Pragmatics,False,10.1162/tacl_a_00586,ACL2023,Disagreement in natural language annotation ha...,"[-0.7916722, 0.70985186, -0.15286492, -0.73208..."
9527,T4929,Time-and-Space-Efficient Weighted Deduction,"Semantics: Sentence-level Semantics, Textual I...",False,10.1162/tacl_a_00588,ACL2023,Many NLP algorithms have been described in ter...,"[-0.12778285, 0.20583482, -0.21488501, -0.6670..."


In [115]:
processed_df.to_csv('../data/classifier_data.csv')